<a href="https://colab.research.google.com/github/VietAnhLee/NLP_1920_FinalExam/blob/master/ML_5_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp "/content/drive/My Drive/Colab Notebooks/NLP_1920_FinalExam/preprocessed_df.csv" "preprocessed_df.csv"

In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('preprocessed_df.csv', sep='\t')

In [0]:
df

,sentence,type,point
0,áo bao đẹp ạ positive,POS,5
1,tuyệt vời positive,POS,5
2,2day ao không giong trong,NEG,1
3,mùi thơm bôi lên da mềm da nagative,POS,5
4,vải đẹp dày dặn positive,POS,5
...,...,...,...
31455,không đáng tiền,NEG,1
31456,quần rất đẹp positive,POS,5
31457,hàng đẹp đúng giá tiền positive positive,POS,5
31458,chất vải khá ổn positive,POS,4


In [0]:
# Try predict only POS, NEU and NEG
df2 = df[['sentence', 'type']].copy()

In [0]:
# Try predict with 5 labels
df2 = df[['sentence', 'point']].copy()

In [0]:
df2

,sentence,point
0,áo bao đẹp ạ positive,5
1,tuyệt vời positive,5
2,2day ao không giong trong,1
3,mùi thơm bôi lên da mềm da nagative,5
4,vải đẹp dày dặn positive,5
...,...,...
31455,không đáng tiền,1
31456,quần rất đẹp positive,5
31457,hàng đẹp đúng giá tiền positive positive,5
31458,chất vải khá ổn positive,4


In [0]:
# convert NEG, NEU, POS to integer
# Use for 3 label version
Dict = {'NEG': '1', 'NEU': '2', 'POS': '3'}
for i in range(len(df2)):
  df2.loc[i, 'type'] = Dict[df2.loc[i, 'type']]  

In [0]:
df2.head()

,sentence,point
0,áo bao đẹp ạ positive,5
1,tuyệt vời positive,5
2,2day ao không giong trong,1
3,mùi thơm bôi lên da mềm da nagative,5
4,vải đẹp dày dặn positive,5


In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df2, test_size=0.3)

In [0]:
train

,sentence,point
16660,sẽ ủng hộ cửa hàng dài dài,4
29416,giao hang rat nhanh,5
7259,chất lượng sản phẩm tuyệt vời đóng gói sản phẩ...,5
21380,chất lượng sản phẩm tuyệt vời cửa hàng phục vụ...,5
27297,nhìn hình thức giống trên hình,5
...,...,...
25798,chất lượng sản phẩm tuyệt vời chất lượng sản p...,5
3227,nước ẩm quá cửa hàng ạ ghi vào là rách giấy na...,3
24960,quần đẹp nhưng đường may notpos lắm lấy 2 cỡ ...,2
8616,chất lượng được positive positive,4


In [0]:
len(train) + len(test)

31460

In [0]:
train.to_csv('train_5label.csv', index=False)
test.to_csv('test_5label.csv', index=False)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [0]:
# using tf-idf with 1 - 3 ngram range
tfidf = TfidfVectorizer(
    min_df = 3, 
    max_df = 0.5, 
    ngram_range=(1,3),
    use_idf=True,
    sublinear_tf=True
)

In [0]:
# Have to add .values.astype('U') to convert type

X_train_tfidf = tfidf.fit_transform(train['sentence'].values.astype('U'))
X_test_tfidf = tfidf.transform(test['sentence'].values.astype('U'))

# v = TfidfVectorizer(decode_error='replace', encoding='utf-8')
# x = v.fit_transform(df['Review'])
# x = v.fit_transform(df['Review'].values.astype('U'))

In [0]:
X_train_tfidf.shape

(22022, 16652)

In [0]:
EXCLUED_COLS = ['sentence', 'point']
static_cols = [c for c in train.columns if not c in EXCLUED_COLS]
X_train_static = train[static_cols].values
X_test_static = test[static_cols].values

In [0]:
train[static_cols].head()

""
16660
29416
7259
21380
27297


In [0]:
from scipy.sparse import hstack, csr_matrix, vstack

In [0]:
X_train = hstack([X_train_tfidf, csr_matrix(X_train_static)]).tocsr()
X_test = hstack([X_test_tfidf, csr_matrix(X_test_static)]).tocsr()

In [0]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [0]:
clfs = {
    "RF": RandomForestClassifier(n_estimators=200, min_samples_leaf=2, max_depth=7, max_features=0.7, random_state=42, n_jobs=-1),
    "GB": GradientBoostingClassifier(n_estimators=200, min_samples_split=2, max_depth=5, max_features=0.7, random_state=111),
    "LR": LogisticRegression(penalty='l2', C=1.0),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM": SVC(C=5, gamma=0.5),
    "SVM-rbf": SVC(C=1000, gamma=0.01, probability=True)
}

In [0]:
for name, clf in clfs.items():
    print('------ ', name, ' ------')
    y_true = train['point'].astype('int')
    clf.fit(X_train, y_true)
    y_pred = clf.predict(X_train)
    
    print("F1", f1_score(y_true, y_pred, average="micro"))
    print("CM", confusion_matrix(y_true, y_pred))
    print("ACC", accuracy_score(y_true, y_pred))
    
    y_true = test['point'].astype('int')
    y_pred = clf.predict(X_test)
    
    print("F1", f1_score(y_true, y_pred, average="micro"))
    print("CM", confusion_matrix(y_true, y_pred))
    print("ACC", accuracy_score(y_true, y_pred))

------  RF  ------
F1 0.538189083643629
CM [[1453  130  110   17  723]
 [ 794  331  285   72  691]
 [ 635  138  561  209 1709]
 [ 225   46  229  606 3728]
 [ 228   17   83  101 8901]]
ACC 0.538189083643629
F1 0.5182242000423819
CM [[ 626   82   42    9  337]
 [ 363  136  129   39  300]
 [ 260   73  206  132  775]
 [ 103   21  113  182 1554]
 [  91    7   35   82 3741]]
ACC 0.5182242000423819
------  GB  ------
F1 0.7429388793025157
CM [[1966  119  108   54  186]
 [ 213 1436  208  142  174]
 [ 176  179 1869  379  649]
 [  58   61  231 2227 2257]
 [  50   22   81  314 8863]]
ACC 0.7429388793025157
F1 0.5595465140919686
CM [[ 620  181  118   51  126]
 [ 242  306  240   81   98]
 [ 130  191  435  322  368]
 [  43   61  197  473 1199]
 [  33   32   82  362 3447]]
ACC 0.5595465140919686
------  LR  ------


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


F1 0.7095631641086186
CM [[1922  170  163   47  131]
 [ 346 1184  360  138  145]
 [ 182  171 1859  394  646]
 [  61   74  312 1904 2483]
 [  55   34  139  345 8757]]
ACC 0.7095631641086186
F1 0.5775588048315321
CM [[ 711  146  120   34   85]
 [ 281  274  259   78   75]
 [ 145  161  494  308  338]
 [  35   61  197  453 1227]
 [  33   11   93  300 3519]]
ACC 0.5775588048315321
------  KNN  ------
F1 0.5913177731359549
CM [[1225   88  534  239  347]
 [ 276  624  642  247  384]
 [ 139  124 2018  476  495]
 [ 106   60  957 2281 1430]
 [  91   15  993 1357 6874]]
ACC 0.5913177731359549
F1 0.4319771137952956
CM [[ 324   65  368  142  197]
 [ 148  149  365  143  162]
 [  92   78  587  309  380]
 [  69   34  496  546  828]
 [  74   10  655  746 2471]]
ACC 0.43197711379529563
------  SVM  ------
F1 0.9179002815366452
CM [[2358   27   21    4   23]
 [  34 2089   29    9   12]
 [  15   33 2897   91  216]
 [   6   11   70 3776  971]
 [   7    4   47  178 9094]]
ACC 0.9179002815366452
F1 0.557003602

In [0]:
# Try with fold
# Not yet install

# from pystacknet.pystacknet import StackNetClassifier

# model = StackNetClassifier(
#     models, metric="accuracy", 
#     folds=5,
#     restacking=False, 
#     use_retraining=True, 
#     use_proba=True, 
#     random_state=12345, n_jobs=1, verbose=1
# )

# y_train = train['label'].astype('int')
# model.fit(X_train, y_train)
# preds=model.predict_proba(X_test)

In [0]:
import pickle

In [0]:
# with open('NLP1920_5Label_.pkl', 'wb') as f:
#   pickle.dump(clfs["RF"], f)

with open('NLP1920_5Label_gradient_boosting.pkl', 'wb') as f:
  pickle.dump(clfs["GB"], f)

with open('NLP1920_5Label_linear_regression.pkl', 'wb') as f:
  pickle.dump(clfs["LR"], f)

with open('NLP1920_5Label_svm.pkl', 'wb') as f:
  pickle.dump(clfs["SVM"], f)

In [0]:
# !cp NLP1920_5Label_random_forest.pkl drive/My\ 'Drive/Colab Notebooks/NLP_1920_FinalExam/ML Models 5 labels'
!cp NLP1920_5Label_gradient_boosting.pkl drive/My\ 'Drive/Colab Notebooks/NLP_1920_FinalExam/ML Models 5 labels'
!cp NLP1920_5Label_linear_regression.pkl drive/My\ 'Drive/Colab Notebooks/NLP_1920_FinalExam/ML Models 5 labels'
!cp NLP1920_5Label_svm.pkl drive/My\ 'Drive/Colab Notebooks/NLP_1920_FinalExam/ML Models 5 labels'

In [0]:
with open('NLP1920_5label_tfidf.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

In [0]:
!cp NLP1920_5label_tfidf.pkl drive/My\ 'Drive/Colab Notebooks/NLP_1920_FinalExam/ML Models 5 labels'

In [0]:
# Test model
# Try with Gradient Boosting first

test_sentence = 'Mỗi tội đóng gói k đc đẹp cho lắm nhưng vẫn cho shop 5 sao.'
sen_tfidf = tfidf.transform([test_sentence])

In [0]:
feat = hstack([sen_tfidf]).tocsr()

In [0]:
label = clfs["SVM"].predict(feat)

In [0]:
# convertToTypeDict = {1: 'NEG', 2: 'NEU', 3: 'POS'}
print(int(label[0]))

5


In [0]:
# !cp test.csv drive/My\ 'Drive/Colab Notebooks/NLP_1920_FinalExam/ML Models'
# !cp train.csv drive/My\ 'Drive/Colab Notebooks/NLP_1920_FinalExam/ML Models'